# Plot Types

This notebook demonstrates all the plot types available in xarray_plotly.

In [ ]:
import plotly.express as px
import xarray as xr

from xarray_plotly import config, xpx

config.notebook()  # Configure Plotly for notebook rendering

## Load Sample Data

We'll use plotly's built-in datasets converted to xarray:
- **stocks**: Tech company stock prices over time
- **gapminder**: Country statistics (life expectancy, GDP, population) by year

In [ ]:
# Stock prices: 2D (date, company)
df_stocks = px.data.stocks().set_index("date")
df_stocks.index = df_stocks.index.astype("datetime64[ns]")

stocks = xr.DataArray(
    df_stocks.values,
    dims=["date", "company"],
    coords={"date": df_stocks.index, "company": df_stocks.columns.tolist()},
    name="price",
)
print(f"stocks: {dict(stocks.sizes)}")

In [ ]:
# Gapminder: pivot to create multi-dimensional arrays
df_gap = px.data.gapminder()

# Life expectancy: 2D (year, country) - select a few countries
countries = ["United States", "China", "Germany", "Brazil", "Nigeria"]
df_life = df_gap[df_gap["country"].isin(countries)].pivot(
    index="year", columns="country", values="lifeExp"
)

life_exp = xr.DataArray(
    df_life.values,
    dims=["year", "country"],
    coords={"year": df_life.index, "country": df_life.columns.tolist()},
    name="life_expectancy",
    attrs={"units": "years"},
)
print(f"life_exp: {dict(life_exp.sizes)}")

In [ ]:
# GDP per capita by continent and year (aggregated)
df_continent = df_gap.groupby(["continent", "year"])["gdpPercap"].mean().reset_index()
df_gdp = df_continent.pivot(index="year", columns="continent", values="gdpPercap")

gdp = xr.DataArray(
    df_gdp.values,
    dims=["year", "continent"],
    coords={"year": df_gdp.index, "continent": df_gdp.columns.tolist()},
    name="gdp_per_capita",
    attrs={"units": "USD"},
)
print(f"gdp: {dict(gdp.sizes)}")

## Line Plot

Best for time series and continuous data:

In [ ]:
fig = xpx(stocks).line(title="Stock Prices Over Time")
fig

In [ ]:
fig = xpx(life_exp).line(title="Life Expectancy by Country", markers=True)
fig

## Bar Chart

Best for comparing categorical data:

In [ ]:
# GDP by continent for selected years
fig = xpx(gdp.sel(year=[1952, 1977, 2007])).bar(title="GDP per Capita by Continent")
fig

### Grouped bars

In [ ]:
fig = xpx(gdp.sel(year=[1952, 1977, 2007])).bar(barmode="group", title="GDP per Capita (Grouped)")
fig

## Area Chart

Best for showing composition over time:

In [ ]:
# Population by continent over time
df_pop = df_gap.groupby(["continent", "year"])["pop"].sum().reset_index()
df_pop_pivot = df_pop.pivot(index="year", columns="continent", values="pop")

population = xr.DataArray(
    df_pop_pivot.values / 1e9,  # Convert to billions
    dims=["year", "continent"],
    coords={"year": df_pop_pivot.index, "continent": df_pop_pivot.columns.tolist()},
    name="population",
    attrs={"units": "billions"},
)

fig = xpx(population).area(title="World Population by Continent")
fig

## Scatter Plot

Best for showing relationships between variables:

In [ ]:
fig = xpx(stocks).scatter(title="Stock Prices Scatter")
fig

## Box Plot

Best for showing distributions:

In [ ]:
# Stock price distributions by company
fig = xpx(stocks).box(title="Stock Price Distributions")
fig

In [ ]:
# Life expectancy distributions by country
fig = xpx(life_exp).box(title="Life Expectancy Distribution by Country")
fig

## Heatmap (imshow)

Best for 2D grid data:

In [ ]:
fig = xpx(life_exp).imshow(title="Life Expectancy Heatmap")
fig

### With different colorscale

In [ ]:
fig = xpx(gdp).imshow(
    color_continuous_scale="Viridis",
    title="GDP per Capita by Continent and Year",
)
fig

## Faceting

All plot types support faceting to create subplot grids:

In [ ]:
# Create 3D data: life expectancy by year, country, and metric
# We'll add GDP as another "metric" dimension
df_metrics = df_gap[df_gap["country"].isin(countries)].pivot(
    index="year", columns="country", values="gdpPercap"
)
gdp_countries = xr.DataArray(
    df_metrics.values,
    dims=["year", "country"],
    coords={"year": df_metrics.index, "country": df_metrics.columns.tolist()},
    name="gdp_per_capita",
)

# Combine into 3D array
combined = xr.concat(
    [life_exp, gdp_countries / 1000],  # Scale GDP to thousands
    dim=xr.Variable("metric", ["Life Exp (years)", "GDP (thousands USD)"]),
)

fig = xpx(combined).line(
    facet_col="metric",
    title="Country Comparison: Life Expectancy and GDP",
)
fig

## Animation

Create animated plots by assigning a dimension to `animation_frame`:

In [ ]:
# Animated bar chart of GDP by continent over time
fig = xpx(gdp).bar(
    x="continent",
    animation_frame="year",
    title="GDP per Capita by Continent (Animated)",
    range_y=[0, 35000],
)
fig

In [ ]:
# Animated line showing life expectancy evolution
fig = xpx(life_exp).bar(
    x="country",
    animation_frame="year",
    title="Life Expectancy by Country (Animated)",
    range_y=[0, 85],
)
fig